<a href="https://colab.research.google.com/github/Multi-Objective-NAS/invertible-nas-algorithm/blob/master/Neural_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [113]:
pip install git+https://github.com/rtqichen/torchdiffeq

  Cloning https://github.com/rtqichen/torchdiffeq to /tmp/pip-req-build-0x8w4l0q
  Running command git clone -q https://github.com/rtqichen/torchdiffeq /tmp/pip-req-build-0x8w4l0q
  Created wheel for torchdiffeq: filename=torchdiffeq-0.0.1-cp36-none-any.whl size=27578 sha256=ef58b071f770357d6aad906adc683898ca625bd85f168f07bd40e230e4d8d310
  Stored in directory: /tmp/pip-ephem-wheel-cache-qnn21__e/wheels/3f/76/69/01867bf3355c3bc8bae7e556b17b44c395b6cda5e76fd8ddc7
Successfully built torchdiffeq


In [114]:
!git clone https://github.com/rtqichen/torchdiffeq

fatal: destination path 'torchdiffeq' already exists and is not an empty directory.


In [115]:
cd torchdiffeq/_impl

[Errno 2] No such file or directory: 'torchdiffeq/_impl'
/content


#Installation dataset

In [116]:
!curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

# Clone and install the code and dependencies.

!git clone https://github.com/google-research/nasbench
!pip install ./nasbench

# Initialize the NASBench object which parses the raw data into memory (this
# should only be run once as it takes up to a few minutes).

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1989M  100 1989M    0     0  87.2M      0  0:00:22  0:00:22 --:--:-- 36.8M
fatal: destination path 'nasbench' already exists and is not an empty directory.
Processing ./nasbench
  Created wheel for nasbench: filename=nasbench-1.0-cp36-none-any.whl size=46787 sha256=94fb740ada75f8b3876c46acee5ec874bb8b7d351aadcbbda78348b613d6e59e
  Stored in directory: /tmp/pip-ephem-wheel-cache-7sw9ua25/wheels/4b/19/99/1d5fdfe30f8b16fab91e900808f4f7e5adc38e602c84970ad5
Successfully built nasbench
  Found existing installation: nasbench 1.0
    Uninstalling nasbench-1.0:
      Successfully uninstalled nasbench-1.0


#Setting

In [117]:
!pwd

/content


In [0]:
import os
import argparse
import logging
import time
import numpy as np
import torch
import torch.nn as nn
import random
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [0]:
from torchdiffeq import odeint

In [0]:
# main argument
#device = torch.device('cuda:0')
device = torch.device('cpu')
is_odenet = True
downsampling_method = 'conv'
batch_size = 100
lr = 0.1
nepochs = 5
rtol=1e-7
atol=1e-9
edge_num = 9

In [121]:
from nasbench import api
nasbench = api.NASBench('nasbench_full.tfrecord')

Loading dataset from file... This may take a few minutes...
Loaded dataset in 315 seconds


In [0]:
# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
OUTPUT = 'output'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3, OUTPUT]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix

#Code

In [0]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def norm(dim):
    return nn.GroupNorm(min(32, dim), dim)

In [0]:
class ODEfunc(nn.Module):

    def __init__(self, dim):
        super(ODEfunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(dim, 50),
            nn.Tanh(),
            nn.Linear(50, dim),
        )
        self.nfe = 0

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, x):
        self.nfe += 1
        return self.net(x)

In [0]:
class ODEBlock(nn.Module):

    def __init__(self, odefunc):
        super(ODEBlock, self).__init__()
        self.odefunc = odefunc
        self.integration_time = torch.tensor([0, 1]).float()

    def forward(self, x):
        self.integration_time = self.integration_time.type_as(x)
        out = odeint(self.odefunc, x, self.integration_time, rtol=rtol, atol=atol)
        return out[1]

    @property
    def nfe(self):
        return self.odefunc.nfe

    @nfe.setter
    def nfe(self, value):
        self.odefunc.nfe = value

##odeint
dy/dt = func(t,y)

y(t[0]) = y0 <- x

method is None:

solution = 'dopri5'.integrate(t)

## integrate method
* _assert_increasing(t)

    * t must be increasing

* before_integrate(t)

    * _select_initial_step
    * rk_state = _RungeKuttaState(y0, f0, t0, t0, first_step, )

* Repeat as len(t):
* solution <= advance(t[i])
    * Until t > rk_state.t1:
    * rk_state = _adaptive_dopri5_step(rk_state)
    * Return, _interp_evaluate(rk_state.interp_coeff, rk_state.t0, rk_state.t1, next_t)

## adaptive dopri5 step










In [126]:
integration_time = torch.tensor([0, 1]).float()
print(integration_time)

tensor([0., 1.])


In [0]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)


In [0]:
class RunningAverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, momentum=0.99):
        self.momentum = momentum
        self.reset()

    def reset(self):
        self.val = None
        self.avg = 0

    def update(self, val):
        if self.val is None:
            self.avg = val
        else:
            self.avg = self.avg * self.momentum + val * (1 - self.momentum)
        self.val = val


In [0]:
def encode(mat, ops):
    def _sanity_check(idx1, opidx, idx2):
        if idx1 not in range(7):
            return False
        if idx2 not in range(7):
            return False
        if opidx not in range(4):
            return False
        return True

    encoded = []
    for inbound, outbound in zip(*np.array(mat).nonzero()):
        op = ALLOWED_OPS.index(ops[outbound])
        assert _sanity_check(inbound, op, outbound)

        embed = [0] * (7 + 7 + len(ALLOWED_OPS))
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1

        encoded += embed

    assert len(encoded) == 9 * (7 + 7 + len(ALLOWED_OPS))
    return encoded

def max_index(output):
    def find_max_index(ls):
        return ls.index(max(ls))

    encoded = []
    embed_size = 7 + 7 + len(ALLOWED_OPS)
    outputs = [output[i: i + embed_size] for i in range(0,len(output), embed_size)]
    for output in outputs:
        embed = [0 for _ in range(embed_size)]
        inbound, op, outbound = find_max_index(output[:7]), find_max_index(output[7:-7]), find_max_index(output[-7:])
        embed[inbound] = 1
        embed[7 + op] = 1
        embed[7 + len(ALLOWED_OPS) + outbound] = 1
        encoded += embed
    
    assert np.sum(np.array(encoded)) == 3
    return encoded

def decode(encoded):
    mat = np.zeros((7,7))
    op = [INPUT] + [CONV3X3 for _ in range(5)] + [OUTPUT]

    embed_size = 7 + 7 + len(ALLOWED_OPS)
    outputs = [output[i: i + embed_size] for i in range(0,len(output), embed_size)]
    for embed in outputs:
        inbound, op, outbound = np.nonzero(np.array(embed))[0]
        op -= 7
        outbound -= (7+len(ALLOWED_OPS))
        mat[inbound][outbound] = 1
        op[outboung] = ALLOWED_OPS[op]
    
    return mat.tolist(), op

In [146]:
y = np.array([0,1,0,1,0])
print(y, np.nonzero(y))
a,b = np.nonzero(y)[0]
print(a,b)

[0 1 0 1 0] (array([1, 3]),)
1 3


In [0]:
#data_loader: x = embedding code, y = (accuracy, latency)
#return train_loader, test_loader, train_eval_loader
def generate_data(batch_size=100, validation_ratio=0.1, test_ratio=0.1):
    def run_experiment(mat, op):
        model_spec = api.ModelSpec(matrix=mat, ops=op)
        q = nasbench.query(model_spec)

        return (q['validation_accuracy'], q['training_time'])
    
    def train_val_test_split(dataset):
        random.shuffle(dataset)
        total_size = (len(dataset) // batch_size) * batch_size
        dataset = dataset[: total_size]
        val_size = int(total_size * validation_ratio)
        test_size = int(total_size * test_ratio)

        val, dataset = dataset[:val_size], dataset[val_size:]
        test, train = dataset[:test_size], dataset[test_size:]

        return train, val, test
    
    def is_target_graph(mat, op):
        return np.sum(mat) == edge_num


    # Find every possible graph
    dataset = []
    for hash_val in nasbench.hash_iterator():
        fixed_stat, _ = nasbench.get_metrics_from_hash(hash_val)
        mat, op = fixed_stat['module_adjacency'], fixed_stat['module_operations']
        if is_target_graph(mat, op):
            x = encode(mat, op)
            y = run_experiment(mat, op)
            dataset.append((x,y))

    return train_val_test_split(dataset)

def data_loader(dataset, batch_turn, size):
    xset = []
    yset = []
    for x,y in dataset[size * batch_turn : size * batch_turn + size]:
        xset.append(x)
        yset.append(y)
    xset = torch.FloatTensor(xset)
    yset = torch.FloatTensor(yset)

In [131]:
x = [100]
y = (10,1)
x = torch.FloatTensor(x)
y = torch.FloatTensor(y)
dataset = []
dataset.append((x,y))
print(dataset)
print(dataset[0])

SyntaxError: ignored

In [0]:
def inf_generator(iterable):
    """Allows training with DataLoaders in a single infinite loop:
        for i, (x, y) in enumerate(inf_generator(train_loader)):
    """
    iterator = iterable.__iter__()
    while True:
        try:
            yield iterator.__next__()
        except StopIteration:
            iterator = iterable.__iter__()


def learning_rate_with_decay(batch_size, batch_denom, batches_per_epoch, boundary_epochs, decay_rates):
    initial_learning_rate = lr * batch_size / batch_denom

    boundaries = [int(batches_per_epoch * epoch) for epoch in boundary_epochs]
    vals = [initial_learning_rate * decay for decay in decay_rates]

    def learning_rate_fn(itr):
        lt = [itr < b for b in boundaries] + [True]
        i = np.argmax(lt)
        return vals[i]

    return learning_rate_fn

In [0]:
def accuracy(model, dataset):
    total_correct = 0
    x,y = data_loader(dataset=dataset, batch_turn=0, size=len(dataset))
    x = torch.FloatTensor(x)
    x = x.to(device)
    y = y.numpy()
    total_correct +=  np.sum(np.square(model(x).cpu().detach().numpy() - y))
    return total_correct / len(dataset)

In [134]:
'''
a = np.array([1.0, 5.0])
b = np.array([2.0, 4.0])
ans = np.sum(np.square(a-b))
print(ans)
'''

'\na = np.array([1.0, 5.0])\nb = np.array([2.0, 4.0])\nans = np.sum(np.square(a-b))\nprint(ans)\n'

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [136]:
!nvidia-smi

Wed Feb 19 11:13:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    29W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
 if downsampling_method == 'conv':
        downsampling_layers = [
            nn.Conv2d(1, 64, 3, 1),
            norm(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1),
            norm(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1),
        ]
elif downsampling_method == 'res':
        downsampling_layers = [
            nn.Conv2d(1, 64, 3, 1),
            ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
            ResBlock(64, 64, stride=2, downsample=conv1x1(64, 64, 2)),
        ]

train_dataset, test_dataset, train_eval_dataset = generate_data(batch_size)

In [0]:
input_size = 9 * (7 + 7 + len(ALLOWED_OPS))
feature_layers = [ODEBlock(ODEfunc(input_size))]
fc_layers = [nn.Linear(input_size, 2)]
model = nn.Sequential(*feature_layers, *fc_layers).to(device)

criterion = nn.MSELoss().to(device)

batches_per_epoch = len(train_dataset) // batch_size

lr_fn = learning_rate_with_decay(
        batch_size, batch_denom=128, batches_per_epoch=batches_per_epoch, boundary_epochs=[60, 100, 140],
        decay_rates=[1, 0.1, 0.01, 0.001]
)

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

best_acc = 0
batch_time_meter = RunningAverageMeter()
f_nfe_meter = RunningAverageMeter()
b_nfe_meter = RunningAverageMeter()
end = time.time()

# Train

In [139]:
for itr in range(nepochs):
    for batch_turn in range(batches_per_epoch):
        x,y = batch_data_loader(dataset=train_dataset, batch_turn=batch_turn, size=batch_size)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_fn(itr)
        optimizer.zero_grad()
        x = x.to(device)
        y = y.to(device)
        logits = model(x)
        loss = criterion(logits, y)

        if is_odenet:
            nfe_forward = feature_layers[0].nfe
            feature_layers[0].nfe = 0

        loss.backward()
        optimizer.step()

        if is_odenet:
            nfe_backward = feature_layers[0].nfe
            feature_layers[0].nfe = 0

        batch_time_meter.update(time.time() - end)
        if is_odenet:
            f_nfe_meter.update(nfe_forward)
            b_nfe_meter.update(nfe_backward)
        end = time.time()

    batch_num = 0
    random.shuffle(train_dataset)
    with torch.no_grad():
        train_acc = accuracy(model, train_eval_dataset)
        val_acc = accuracy(model, test_dataset)
        '''if val_acc > best_acc:
                torch.save({'state_dict': model.state_dict(), 'args': args}, os.path.join(args.save, 'model.pth'))
                best_acc = val_acc
            '''
        print(
                "Epoch {:04d} | Time {:.3f} ({:.3f}) | NFE-F {:.1f} | NFE-B {:.1f} | "
                Train Acc {:.4f} | Test Acc {:.4f}".format(
                itr // batches_per_epoch, batch_time_meter.val, batch_time_meter.avg, f_nfe_meter.avg,
                b_nfe_meter.avg, train_acc, val_acc
            )

SyntaxError: ignored

# Predict input

adjoint.py의 81번째 코드 참고!

In [0]:
# predict x by y